# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Setting up the database

#### Creating a Cluster

In [5]:

from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Creating a Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Current Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## The queries needed are:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### For the table artist_details, modeled after Query 1, a composite key of session_id and item_in_session was used to make a unique jey. 

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS artist_details"
query = query + "(session_id int, item_in_session int, artist_name text,song_title text, song_length float, PRIMARY KEY (session_id,item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

### The records are added into artist_details in the code snippet below:

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_details (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### A SELECT verified that the data has been inserted into each table

In [10]:
query = "SELECT artist_name, song_title, song_length from artist_details WHERE SESSION_ID=338 AND ITEM_IN_SESSION=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, ',', row.song_title, ',', row.song_length)


Faithless , Music Matters (Mark Knight Dub) , 495.30731201171875


### For Query 2, a table user_song_library was created with a composite key of user_id and session_id, and item_in_session was used as a clustering column so the records would be sorted accordingly.

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query2 = "CREATE TABLE IF NOT EXISTS user_song_library"
query2 = query2 + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id) , item_in_session))"
try:
    session.execute(query2)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query2 = "INSERT INTO user_song_library (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name )"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))                     

### For Query 3, a table user_history was created with a primary key song_title, and clustering columns first_name and last_name

In [12]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query3 = "CREATE TABLE IF NOT EXISTS user_history"
query3 = query3 + "(song_title text, first_name text, last_name text, PRIMARY KEY ((song_title), first_name, last_name))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query3 = "INSERT INTO user_history (song_title, first_name, last_name)"
        query3 = query3 + " VALUES (%s, %s, %s)"
        session.execute(query3, (line[9], line[1], line[4]))            


## SELECT queries for both tables are run below and successfully output records!

In [13]:
query2 = "SELECT artist_name, song_title, first_name, last_name FROM user_song_library WHERE USER_ID=10 AND SESSION_ID = 182"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, ',' , row.song_title,',', row.first_name, row.last_name)


Down To The Bone , Keep On Keepin' On , Sylvie Cruz
Three Drives , Greece 2000 , Sylvie Cruz
Sebastien Tellier , Kilometer , Sylvie Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie Cruz


In [14]:
query3 = "select first_name, last_name from user_history WHERE SONG_TITLE='All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)


Jacqueline Lynch
Sara Johnson
Tegan Levine


### Dropping the tables before closing out the sessions

In [15]:
query1 = "drop table artist_details"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query2 = "drop table user_song_library"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
query3 = "drop table user_history"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)


### Closing the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()